In [2]:
import json
from kafka import KafkaConsumer

if __name__ == "__main__":
    consumer = KafkaConsumer("ftde02-project4", bootstrap_servers='localhost:9093')
    print("Starting the consumer")
    
    for msg in consumer:
        print(f"Records = {json.loads(msg.value)}")
        
        # Parsing pesan Kafka
        data = json.loads(msg.value)

Starting the consumer
Records = {'step': 1, 'type': 'PAYMENT', 'amount': 1157.86, 'nameOrig': 'C1237762639', 'newbalanceOrig': 19998.14, 'nameDest': 'M1877062907', 'newbalanceDest': 0.0}
Records = {'step': 1, 'type': 'PAYMENT', 'amount': 671.64, 'nameOrig': 'C2033524545', 'newbalanceOrig': 14451.36, 'nameDest': 'M473053293', 'newbalanceDest': 0.0}
Records = {'step': 1, 'type': 'TRANSFER', 'amount': 215310.3, 'nameOrig': 'C1670993182', 'newbalanceOrig': 0.0, 'nameDest': 'C1100439041', 'newbalanceDest': 0.0}
Records = {'step': 1, 'type': 'PAYMENT', 'amount': 1373.43, 'nameOrig': 'C20804602', 'newbalanceOrig': 12480.57, 'nameDest': 'M1344519051', 'newbalanceDest': 0.0}
Records = {'step': 1, 'type': 'DEBIT', 'amount': 9302.79, 'nameOrig': 'C1566511282', 'newbalanceOrig': 1996.21, 'nameDest': 'C1973538135', 'newbalanceDest': 16896.7}
Records = {'step': 1, 'type': 'DEBIT', 'amount': 1065.41, 'nameOrig': 'C1959239586', 'newbalanceOrig': 751.59, 'nameDest': 'C515132998', 'newbalanceDest': 0.0}

KeyboardInterrupt: 

In [3]:
data

{'step': 1,
 'type': 'PAYMENT',
 'amount': 7823.46,
 'nameOrig': 'C260084831',
 'newbalanceOrig': 0.0,
 'nameDest': 'M267814113',
 'newbalanceDest': 0.0}

In [4]:
import pandas as pd
import urllib.parse

from sqlalchemy import create_engine

# Informasi koneksi ke PostgreSQL
username = "ftde02"
password = "ftde02!@#"
host = "localhost"
port = "6543"
database = "stream_processing"
password = urllib.parse.quote_plus(password)

# URL koneksi ke PostgreSQL
db_url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url)
print("PostgreSQL DB connection success!")

PostgreSQL DB connection success!


In [5]:
df = pd.read_sql_query('SELECT * FROM old_information;', engine)
df.head(5)

,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest
0,C1231006815,170136.0,M1979787155,0.0
1,C1666544295,21249.0,M2044282225,0.0
2,C1305486145,181.0,C553264065,0.0
3,C840083671,181.0,C38997010,21182.0
4,C2048537720,41554.0,M1230701703,0.0


In [6]:
df.columns

Index(['nameOrig', 'oldbalanceOrg', 'nameDest', 'oldbalanceDest'], dtype='object')

### **Join Data From Producer Kafka and Database PostgreSQL**

In [7]:
produder = pd.DataFrame([data])
produder

,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest
0,1,PAYMENT,7823.46,C260084831,0.0,M267814113,0.0


In [8]:
data = produder.merge(df, how='inner', on=['nameOrig','nameDest'])
predict = data.drop(['nameOrig','nameDest'], axis=1)
predict = predict.to_dict('index')[0]
predict

{'step': 1,
 'type': 'PAYMENT',
 'amount': 7823.46,
 'newbalanceOrig': 0.0,
 'newbalanceDest': 0.0,
 'oldbalanceOrg': 998.0,
 'oldbalanceDest': 0.0}

### **Precit Fraud Detection**

In [9]:
from modelling import FraudModel

In [10]:
#parameter inputan path
import os

path = os.getcwd()
path = path + "\\modelling\\"
path

'c:\\Users\\RickyS-PC\\Desktop\\Training\\Tech-software\\Bootcamp\\Digital-Skola-FS-Data-Engineer\\Class\\mini-project4\\modelling\\'

In [11]:
result = FraudModel.runModel(predict, path)
result

c:\Users\RickyS-PC\miniconda3\envs\ftde-final-project\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\RickyS-PC\miniconda3\envs\ftde-final-project\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\RickyS-PC\miniconda3\envs\ftde-final-project\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator S

'White List'

In [12]:
data['predict'] = result
data

,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest,oldbalanceOrg,oldbalanceDest,predict
0,1,PAYMENT,7823.46,C260084831,0.0,M267814113,0.0,998.0,0.0,White List


### **Inser MongoDB**

In [57]:
from pymongo import MongoClient

# Mengatur koneksi ke MongoDB
mongo_client = MongoClient("mongodb://127.0.0.1:27017")

db = mongo_client["ftde02"]
collection = db["project4-collection"]

if isinstance(data.to_dict('index')[0], list):
    collection.insert_many(data.to_dict('index')[0])
else:
    collection.insert_one(data.to_dict('index')[0])        
    print("Data telah disimpan ke MongoDB")

Data telah disimpan ke MongoDB


In [58]:
mongo_client = MongoClient("mongodb://127.0.0.1:27017")
db = mongo_client["ftde02"]
collection = db["project4-collection"]
    
# Membaca data dari MongoDB ke dalam list of dictionaries
data_from_mongo = list(collection.find())
    
# Membaca data ke dalam DataFrame
df = pd.DataFrame(data_from_mongo)
df

,_id,step,type,amount,nameOrig,newbalanceOrig,nameDest,newbalanceDest,oldbalanceOrg,oldbalanceDest,predict
0,668a24415444906cb867cdba,1,PAYMENT,2131.84,C1987977423,0.0,M61073295,0.0,224.0,0.0,White List
